In [1]:
import h5py
import numpy as np
import matplotlib
matplotlib.use('Agg')
matplotlib.rcParams['mathtext.default'] = 'regular'
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from PIL import Image
import numpy.ma as ma
from tempfile import TemporaryFile
from numpy import *

/Users/litadlc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mp = 1.672622e-24 # mass of hydrogren atom, in grams
kb = 1.380658e-16 # boltzmann constant in ergs/K

d_c = mp # code density unit, 1 mp / cm^3
l_c = 3.08567758e18 # code length unit, 1 pc in cm
l_km = 3.08567758e13 # code length unit, 1pc in km
t_c = 3.15569e10    # code time unit, 1 kyr in seconds
v_c = l_c / t_c     # code velocity unit
v_km = l_km / t_c     # code velocity unit
p_c = d_c * v_c * v_c # code pressure unit
m_c = d_c * l_c * l_c * l_c / 1.9891e33 # code mass unit to solar masses

In [3]:
istart = 398 # starting file number
iend   = 398 # ending file number
dx = 5./64. # size scale, pc per cell
t_cc = 39.8 # cloud crushing time in kyr, n = 0.5 cloud
#t_cc = 56.4 # cloud crushing time in kyr, n = 1 cloud
dnamein='/Users/litadlc/Desktop/Data/' # directory where the file is located
dnameout='/Users/litadlc/Desktop/10tcc/' # directory where the plot will be saved

In [4]:
for i in range(istart,iend+1,30):
    print(i)
f = h5py.File(dnamein+str(i)+'.h5', 'r') # open the hdf5 file for reading
head = f.attrs # read the header attributes into structure, head
gamma = head['gamma'][0] # ratio of specific heats
t  = head['t'][0] # time of this snapshot, in kyr
nx = head['dims'][0] # number of cells in the x direction
ny = head['dims'][1] # number of cells in the y direction
nz = head['dims'][2] # number of cells in the z direction
d  = f['density'][:]
mx = f['momentum_x'][:]
#E = f['Energy'][:]
GE = f['GasEnergy'][:] # internal energy in code units, d*e
f.close()

398


In [5]:
mu = 1.0 # mean molecular weight (mu) of 1
den = d*d_c # to convert from code units to cgs, multiply by the code unit for that variable)
f_h=0.74 #solar metallicity
nFe_nH=2.82e-5
nSi_nH=3.47e-5
nO_nH=4.90e-4
nC_nH=2.45e-4
nAl_nH=2.95e-6
nN_nH=8.51e-5
nS_nH=1.84e-5
#Calculation number density for each particular species
n_Fe = f_h*(den/(mu*mp))*(nFe_nH)
n_Si1 = f_h*(den/(mu*mp))*(nSi_nH)
n_Si2 = f_h*(den/(mu*mp))*(nSi_nH)
n_O1 = f_h*(den/(mu*mp))*(nO_nH)
n_O2 = f_h*(den/(mu*mp))*(nO_nH)
n_C1 = f_h*(den/(mu*mp))*(nC_nH)
n_C2 = f_h*(den/(mu*mp))*(nC_nH)
n_N = f_h*(den/(mu*mp))*(nN_nH)
n_S = f_h*(den/(mu*mp))*(nS_nH)
n_Al = f_h*(den/(mu*mp))*(nAl_nH)

In [6]:
T = (GE*(gamma-1.0)*p_c)/(d*kb)
log_T = np.log10(T)

In [7]:
FeII = ma.masked_outside(log_T,4.05,4.15)
SiII = ma.masked_outside(log_T,3.92,4.08)
SiIV =  ma.masked_outside(log_T,4.30,4.45)
OVI = ma.masked_outside(log_T,5.4,5.5)
OI = ma.masked_outside(log_T,4.0,4.15)
CII = ma.masked_outside(log_T,4.35,4.48)
CIV = ma.masked_outside(log_T,4.95,5.10)
NV =  ma.masked_outside(log_T,5.20,5.35)
SII =  ma.masked_outside(log_T,4.20,4.35)

In [8]:
FeII_mask = ma.getmask(FeII)
SiII_mask = ma.getmask(SiII)
SiIV_mask = ma.getmask(SiIV)
OI_mask = ma.getmask(OI)
OVI_mask = ma.getmask(OVI)
CII_mask = ma.getmask(CII)
CIV_mask = ma.getmask(CIV)
NV_mask = ma.getmask(NV)
SII_mask = ma.getmask(SII)

In [9]:
n_Fe[FeII_mask] = 10e-20
n_Si1[SiII_mask] = 10e-20
n_Si2[SiIV_mask] = 10e-20
n_O1[OI_mask] = 10e-20
n_O2[OVI_mask] = 10e-20
n_C1[CII_mask] = 10e-20
n_C2[CIV_mask] = 10e-20
n_N[NV_mask] = 10e-20
n_S[SII_mask] = 10e-20

In [20]:
#Number density
n_FeII = n_Fe
n_SiII = n_Si1
n_SiIV = n_Si2
n_OI   = n_O1
n_OVI  = n_O2
n_CII  = n_C1
n_CIV  = n_C2
n_NV   = n_N
n_SII  = n_S

In [11]:
#Velocity array
vx = ((mx)/(d))*v_km #in km along the x-axis

In [12]:
#Creating a j,k,vel array
values_SiII=np.zeros((ny,nz,100))
for j in range(0,ny):
    for k in range(0,nz):
        n_SiII_1D=n_SiII[:,j,k]
        vel_SiII=vx[:,j,k]
        values_SiII[j,k,:],b=np.histogram(vel_SiII,bins=100,weights=n_SiII_1D,range=(0,1200))

In [13]:
#Creating a j,k,vel array
values_SiIV=np.zeros((ny,nz,100))
for j in range(0,ny):
    for k in range(0,nz):
        n_SiIV_1D=n_SiIV[:,j,k]
        vel_SiIV=vx[:,j,k]
        values_SiIV[j,k,:],b=np.histogram(vel_SiIV,bins=100,weights=n_SiIV_1D,range=(0,1200))

In [14]:
values_OI=np.zeros((ny,nz,100))
for j in range(0,ny):
    for k in range(0,nz):
        n_OI_1D=n_OI[:,j,k]
        vel_OI=vx[:,j,k]
        values_OI[j,k,:],b=np.histogram(vel_OI,bins=100,weights=n_OI_1D,range=(0,1200))

In [15]:
values_OVI=np.zeros((ny,nz,100))
for j in range(0,ny):
    for k in range(0,nz):
        n_OVI_1D=n_OVI[:,j,k]
        vel_OVI=vx[:,j,k]
        values_OVI[j,k,:],b=np.histogram(vel_OVI,bins=100,weights=n_OVI_1D,range=(0,1200))

In [16]:
values_CII=np.zeros((ny,nz,100))
for j in range(0,ny):
    for k in range(0,nz):
        n_CII_1D=n_CII[:,j,k]
        vel_CII=vx[:,j,k]
        values_CII[j,k,:],b=np.histogram(vel_CII,bins=100,weights=n_CII_1D,range=(0,1200))

In [17]:
values_CIV=np.zeros((ny,nz,100))
for j in range(0,ny):
    for k in range(0,nz):
        n_CIV_1D=n_CIV[:,j,k]
        vel_CIV=vx[:,j,k]
        values_CIV[j,k,:],b=np.histogram(vel_CIV,bins=100,weights=n_CIV_1D,range=(0,1200))

In [18]:
values_NV=np.zeros((ny,nz,100))
for j in range(0,ny):
    for k in range(0,nz):
        n_NV_1D=n_NV[:,j,k]
        vel_NV=vx[:,j,k]
        values_NV[j,k,:],b=np.histogram(vel_NV,bins=100,weights=n_NV_1D,range=(0,1200))

In [21]:
values_SII=np.zeros((ny,nz,100))
for j in range(0,ny):
    for k in range(0,nz):
        n_SII_1D=n_SII[:,j,k]
        vel_SII=vx[:,j,k]
        values_SII[j,k,:],b=np.histogram(vel_SII,bins=100,weights=n_SII_1D,range=(0,1200))

In [22]:
np.save('n_v_SiII',values_SiII)
np.save('n_v_SiIV',values_SiIV)
np.save('n_v_OI',values_OI)
np.save('n_v_OVI',values_OVI)
np.save('n_v_CII',values_CII)
np.save('n_v_CIV',values_CIV)
np.save('n_v_NV',values_NV)
np.save('n_v_SII',values_SII)